# 第一章 `lex`和`yacc`

## `lex`程序

一个典型的lex程序如下所示：

```c
%{
    // header
%}
%%

[\t ]+ ;
is |
am |
go {
        printf("%s: is a verb;\n", yytext);
   }

[a-zA-Z]+ {
              printf("%s: is not a verb", yytext);
          }
%%

int main() {
    yylex();
}
```

* 第一部分：用特殊的界定符号`%{`以及`%}`标记，lex将这一部分的内容直接拷贝到生成的C语言文件中，这一部分可以包含一些后续的C语言代码将会直接使用的部分，例如引用一些后续代码将会引用的头文件，定义一些枚举等等。


* 第二部分：规则段部分，每个规则由两部分组成：
    1. 模式：模式由正则表达式定义。
    2. 动作：当lex识别出某个模式时将执行相应的动作
  
  以上两个部分通过**空格**隔开，多个“规则”对应的“动作”是相同的，那么规则后面可以跟`|`动作，这是一个特殊的动作，意味着和下一个模式使用相同的动作。如果输入的文本能够匹配多个定义的规则，那么lex有一套默认的匹配规则：
    1. lex模式只能匹配输入字符串一次。
    2. lex执行当前输入的最长可能的匹配动作。


* 第三部分：这是用户例程部分，由任意的用户自定义的C语言程序组成，lex将会把用户定义的C语言代码拷贝到最终生成的c代码中。

以上三个部分之间通过特殊符号`%%`来分别标记，在lex程序中，有一些默认的函数和指针可以使用：

* `yytext`：是一个字符指针，当lex匹配到一个模式时，在相应的动作中就可以引用这个指针来指向刚刚匹配到的这个字符串。
* `yylex`：是一个函数，当调用这个函数时lex默认会监听当前的标准输入（即键盘输入），每当输入一行文本按下回车lex程序就会分析刚才输入的文本行。

## `yacc`程序

`yacc`程序是lex程序的更复杂的一层，在词法分析的基础上定义了语法。“语法”可以定义为一组特殊“标记”组成的序列，这些标记即lex分析程序输出，简单来讲可以理解为lex匹配到的“模式”，例如我们可以通过lex分析程序通过定义“模式”的方式来界定英语中的哪些单词是动词，哪些是名词。那么现在如果有一句英文：

```text
I love you;
```

我们可以这样定义lex词法分析程序

```c
%{%}
%%

[\t ]+ ;
I |
you {return NOUN;}

love {return VERB;}

;\n {return 0;}

%%

```

那么当我们调用`yylex()`的时候：lex分析程序在分析输入的`I love you;`时就会分别在检测到相应的单词时返回相应的**token**（即：NOUN，VERB，0）。所谓“语法”就是这些token的有序排列组成的集合，yacc就是为了处理这样的token集合而出现的，在yacc中将这样的token有序集合定义为“语法”，并且针对特定的“语法”可以定义出相应的“动作”（就如同lex分析程序针对不同的模式可以定义出不同的动作一样）

### `yacc`程序的组成部分

一个`yacc`程序和`lex`是相似的，也分为三个部分，第一部分和第三部分和lex分析程序相同，重点是第二部分，即规则的定义部分和lex有所不同，yacc语法分析程序在第二部分进行具体的语法定义：

```c
%{

%}

%token NOUN VERB
%%

sentence: simple_sentence {printf("This is a simple sentence;");}
          | compound_sentence {printf("This is a compound sentence");}
          ;
simple_sentence: NOUN VERB NOUN {printf("This is token sequneces;");}
                ;
compound_sentence: simple_sentence simple_sentence {printf("This is compound;");}
                ;
%%

extern FILE *yyin;
int main() {
    do {
        yyparse();
    } while (!feof(yyin))
}

void yyerror(char *s) {
    fprintf(stderr, "Error: %s", s);
}
```

在第二部分规则定义中，yacc可以定义标记（就像lex一样），所不同的是：

* yacc的标记（即语法）可以是由若干token组成，也可以是由其他的yacc标记组成，或者是其他yacc标记以及token组成的集合
* 一个yacc标记可以由多个“定义”，即一个sentence既可以是一个simple_sentence也可以是一个compound_sentence
* 一个yacc标记可以递归定义

针对lex而言，每个规则只能定义一个动作。但是在yacc中由于一个标记可以有多个“定义”，因此每个“定义”都能定义动作，不同的“定义”之间使用特殊符号`|`连接，最后以`;`结尾。

### `yacc`的内置函数

yacc和lex一样也拥有内置函数，在本例中就是yyparse，这个函数需要调用lex分析程序中的yylex来首先进行词法分析，获取输入文本中的token集合，根据这些token集合再去匹配相应的语法规则。在yacc的定义文件中可以通过`%token`这样的特殊写法来定义token，在lex文件中通过`#include "y.tab.h"`来感知yacc定义文件中的这些token。

在实际的分析过程中，lex需要告诉yacc“什么时候一句话结尾了”。换句话说，如果现在有很多英文句子：
```text
I love you; you love me; I play games;
```
yacc其实并不知道什么时候断句，然后将一整句话进行语法分析，这个“断句”的信息其实是lex给出的，lex会通过返回一个特殊的token（即0）告诉yacc程序当前的句子已经结束，可以根据之前返回的token序列进行语法分析了。

举一个具体的例子，假设现在有三句话（每句话以分号和换行结尾）：

```text
I love you;
you love me;
```

此时调用`yyparse`，函数内部会先调用yylex进行语法分析，yylex首先读到I，然后返回NOUN，然后依次返回VERB和NOUN，最后读取到`;\n`返回0告诉yacc程序当前句子已经结束，yacc就会根据lex之前返回的`NOUN VERB NOUN`进行语法匹配，匹配到simple_sentence这个语法规则，然后制定语法文件中定义的动作，此时yyparse也会返回，此次调用就结束。

### yacc与lex的通信

根据上面的表述可以知道，yacc需要和lex配合才能够正常工作，毕竟yacc的语法规则本身就是lex标记的序列集合。它们之间的通信依靠的是`y.tab.h`这个头文件完成的，这个头文件是yacc自动生成的，通过在lex中引用这个头文件可以引用到yacc中定义的一些内容，例如`%token`定义。

# 第二章 使用`lex`

`lex`是词法分析的核心，它既可以与`yacc`一起使用，作为`yacc`的上游输入，也可以单独使用。`lex`的核心是正则表达式，它使用正则表达式作为匹配单词识别token的基本工具。

## 统计词频的工具

统计词频是`lex`的一个常见的应用场景，在这个例子中可以见到`lex`的一些特殊用法，也会引入和`lex`相关的另一个内置函数`yywrap`。以下是这个场景的一个非常简单的实现：

```c
%{
    unsigned word_count = 0; // 单词数目
    unsigned char_count = 0; // 字符数目
    unsigned line_count = 0; // 行数
%}

word [^ \t\n]+
eol \n
%%

{word} {word_count ++; char_count += yyleng}
{eol} {char_count++; line_count++}
. {char_count++;}
%%
    
int main() {
    yylex();
    printf("%15d, %15d, %15d\n", word_count, char_count, line_count);
    return 0;
}
```

这个例子中和前一章的`lex`例子不同之处在于，前一章的`lex`规则（即正则表达式）都是定义在规则段当中的，这个例子中的规则在第一部分中就使用了标识符来指代，这样就不用在规则部分重复定义复杂的正则表达式了，实际上是将规则能够复用。

在这个例子中使用了一个内置变量`yyleng`用以指代`lex`匹配到的单词`yytext`的长度，这个例子是比较常规的，它依然是从stdin即标准输入流中读取字符，下面这个例子将会从文件流中读入内容：

```c
%{
    // 单文件输入词频统计lex词法分析程序，只接受单个文件的输入
    unsigned word_count = 0; // 单词数统计
    unsigned line_count = 0; // 行数统计
    unsigned char_count = 0; // 字符数统计
%}

word_reg [^ \t\n]+
eol_reg \n
%%

{word_reg} {
    word_count ++;
    char_count += yyleng;
}

{eol_reg} {
    char_count ++;
    line_count ++;
}

. {
    char_count ++;
}
%%

/*
简单词频统计函数的入口，这个函数只能统计单个输入文件的词频

@param argc: 命令行输入时参数的个数
@param argv: 命令行输入时参数的具体值
*/
int simple_word_count(int argc, char **args) {
    if (argc == 2) {
        char *file_path = args[1];
        FILE *fp = fopen(file_path, "r");
        if (!fp) {
            fprintf(stderr, "can not open file \"%s\"\n", file_path);
            return 1;
        }
        yyin = fp;
    }
    yylex();
    printf("word_count: %d, line_count: %d, char_count: %d\n", word_count, line_count, char_count);
    close(yyin);
    return 0;
}
```

上面这个例子唯一的改进之处在于它允许动态地从文件中读取内容作为`lex`词法分析的输入。

不过上面的两个例子都有一个弊端：如果我有多个文件要想一并读取怎么办呢？`yylex()`在进行读取的过程中会不断从`yyin`中读取，如果`yyin`是一个文件输入，那么当`yylex()`读取到文件的末尾时会直接返回0，如果要支持多个文件的词频统计就必须调用多次`yylex()`，并且在每次调用之前都必须重新将`yyin`重新指向新的输入文件。

上述的方法未免有些复杂，在`lex`中提供了一个`yywarp()`函数，这个函数并不由使用者来调用，而是通过`yylex()`内部调用的，当yylex扫描到文件末尾时会调用`yywrap()`，如果`yywarp`:

* 返回1:这是默认返回，说明已经没有更多的文件需要读取了，`yylex`将会返回0，结束调用。
* 返回0:这等于告诉`yylex`还有别的输入没有处理，此时`yywarp`需要重新将`yyin`重定向到新的文件上然后返回0，获取到0返回的`yylex`会重新从`yyin`中读取。

`yywrap`是`lex`的一个内置函数（或者宏，有的lex会把`yywrap`实现为一个宏），在需要使用`yywrap`时需要重新定义`yywrap`，这和`yyerror(char *)`有点相似，一个简单的例子如下：

```c
%{
    // 单文件输入词频统计lex词法分析程序，只接受单个文件的输入
    unsigned word_count = 0; // 单词数统计
    unsigned line_count = 0; // 行数统计
    unsigned char_count = 0; // 字符数统计
%}

word_reg [^ \t\n]+
eol_reg \n
%%

{word_reg} {
    word_count ++;
    char_count += yyleng;
}

{eol_reg} {
    char_count ++;
    line_count ++;
}

. {
    char_count ++;
}
%%

static int total_exec = 2;
static int outer_argc;
static char** outer_argv;

/*
简单词频统计函数的入口，这个函数只能统计单个输入文件的词频

@param argc: 命令行输入时参数的个数
@param argv: 命令行输入时参数的具体值
*/
int simple_word_count(int argc, char **args) {
    outer_argc = argc;
    outer_argv = args;
    int return_val = yywrap();
    if (return_val == -1) {
        return return_val;
    }
    yylex();
    printf("word_count: %d, line_count: %d, char_count: %d\n", word_count, line_count, char_count);
    close(yyin);
    return 0;
}

int yywrap() {
    if (outer_argc == 2) {
        char *file_path = outer_argv[1];
        close(yyin);
        FILE *fp = fopen(file_path, "r");
        if (!fp) {
            fprintf(stderr, "can not open file \"%s\"\n", file_path);
            return -1;
        }
        yyin = fp;
    }
    if ((--total_exec) >= 0) {
        return 0;
    }
    return 1;
}
```

上面这个例子可以看作是之前例子的拓展，它使用了`total_exec`用于控制`yywrap`到底是输出1还是0，在这个例子中所要达到的效果就是将目标`yyin`词法解析两次，最终运行的效果也是我们预期的：

```shell
yh-swjtudeMacBook-Pro:build yh_swjtu$ ./yacc /Users/yh_swjtu/Desktop/yacc/examples/sonnet.txt
Begin...
word_count: 216, line_count: 28, char_count: 1268
```

## 分析命令行

这个例子很特殊，它第一次引入了**状态**这个概念。这让lex分析程序第一次有了“状态机”的味道，这里要介绍一个很重要的lex语法，用于标记一个状态：

```c
%{
    // 带有状态的词法分析，可以理解为自动机的状态，这是一个解析命令行中参数的例子
    // %s 在此处用于定义一个状态
%}
%s FNAME
%%

[ \n\t]+ ;

-h |
--help {
    printf("this is help message\n");
}

-v |
--verbose {
    printf("This is verbose message\n");
}

-f |
--file {
    printf("this is file messgae\n");
    BEGIN FNAME;
}

<FNAME>[a-zA-Z\\.]+ {
    printf("file name: %s\n", yytext);
    BEGIN 0;
}

[^ ]+ {
    printf("ECHO: %s\n", yytext);
}
%%

void parse_command_line() {
    yylex();
}
```

在这里例子中可以看出在lex词法文件的头部使用了特殊语法`%s FNAME`来定义一个特殊的状态，这个状态就像自动机中的状态一样。在规则定义阶段，如果检测到了`-f`或者`--file`那么就开启这个状态：`BEGIN FNAME;`，在规则定义部分中，使用特殊的语法`<FNAME>`来规定后面的规则定义只有在`FNAME`状态启用的时候才去匹配。

需要说明的是：如果一个规则不定义状态，那么就意味着它在任意状态下都能匹配。上述这个例子由于标准命令行输入的时候的缓冲问题输出可能不符合预期，因此采用文件读入的方式可能会更好一些。